---
<center><h1>Spam Mail Prediction</h1></center>
<center><h3>Part of 30 Days 30 ML Projects Challenge</h3></center>

---

## 1) Understanding Problem Statement
---

In the age of digital communication, spam emails have become a significant nuisance, cluttering inboxes and potentially causing harm. This project addresses the issue of spam email detection through the lens of machine learning and data science.

The problem at hand falls into the category of **Binary Classification Machine Learning Problem**. The central objective is **to create a predictive model capable of distinguishing between spam and legitimate emails**. This model will rely on various email attributes and content features to make informed predictions, ultimately helping users filter out unwanted and potentially harmful messages. Moreover, It also involves use of **Natural Language Processing (NLP)** techniques for handling textual data.

## 2) Understanding Data
---

The project uses **Mail Data** which contains several variables (independent variables) and the outcome variable or dependent variable.

## 3) Getting System Ready
---
Importing required libraries


In [3]:
import numpy as np
import pandas as pd

# for text data preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# for model buidling
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Downloading stop words for text preprocessing

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 4) Data Eyeballing
---

### Laoding Data

In [6]:
mail_data = pd.read_csv('Datasets/Day17_Mail_Data.csv') 

In [7]:
mail_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
print('The size of Dataframe is: ', mail_data.shape)
print('-'*100)
print('The Column Name, Record Count and Data Types are as follows: ')
mail_data.info()
print('-'*100)

The size of Dataframe is:  (5572, 2)
----------------------------------------------------------------------------------------------------
The Column Name, Record Count and Data Types are as follows: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
----------------------------------------------------------------------------------------------------


In [9]:
# Defining numerical & categorical columns
numeric_features = [feature for feature in mail_data.columns if mail_data[feature].dtype != 'O']
categorical_features = [feature for feature in mail_data.columns if mail_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 0 numerical features : []

We have 2 categorical features : ['Category', 'Message']


In [10]:
print('Missing Value Presence in different columns of DataFrame are as follows : ')
print('-'*100)
total=mail_data.isnull().sum().sort_values(ascending=False)
percent=(mail_data.isnull().sum()/mail_data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

Missing Value Presence in different columns of DataFrame are as follows : 
----------------------------------------------------------------------------------------------------


,Total,Percent
Category,0,0.0
Message,0,0.0


In [11]:
print('Summary Statistics of numerical features for DataFrame are as follows:')
print('-'*100)
mail_data.describe(include='object')

Summary Statistics of numerical features for DataFrame are as follows:
----------------------------------------------------------------------------------------------------


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


## 5) Data Cleaning and Preprocessing
---

### Replace the null values with a null string (Only for demonstration, here it is not neede)

In [12]:
mail_data = mail_data.where((pd.notnull(mail_data)),'')

In [13]:
mail_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Label Encoding

#### label spam mail as 0;  ham mail as 1;

In [14]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

In [15]:
mail_data

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [16]:
mail_data['Category'].value_counts()

Category
1    4825
0     747
Name: count, dtype: int64

### Stemming

In [17]:
porter_stemmer = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
mail_data['Message'] = mail_data['Message'].apply(stemming)

In [ ]:
mail

## 6) Model Building
---

### Creating Feature Matrix (Independent Variables) & Target Variable (Dependent Variable)

In [17]:
# separating the data and labels
X = mail_data.drop(columns = ['Category'], axis=1) # Feature matrix
y = mail_data['Category'] # Target variable

In [18]:
X

,Message
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."
...,...
5567,This is the 2nd time we have tried 2 contact u...
5568,Will ü b going to esplanade fr home?
5569,"Pity, * was in mood for that. So...any other s..."
5570,The guy did some bitching but I acted like i'd...


In [19]:
y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object

### Train-Test Split

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [21]:
print(X.shape, X_train.shape, X_test.shape)

(5572, 1) (4457, 1) (1115, 1)


In [22]:
print(y.shape, y_train.shape, y_test.shape)

(5572,) (4457,) (1115,)


### Feature Extraction

#### Transform the text data to feature vectors that can be used as input to the Logistic regression

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)


y_train = y_train.astype('int')
y_test = y_test.astype('int')

InvalidParameterError: The 'lowercase' parameter of TfidfVectorizer must be an instance of 'bool', an instance of 'numpy.bool_' or an instance of 'int'. Got 'True' instead.

In [ ]:

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

### Model Comparison : Training & Evaluation

In [37]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

In [38]:
classification_metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

classification_metrics_df.set_index('Model', inplace=True)
classification_metrics_df

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Logistic Regression,0.798883,0.800000,0.637681,0.709677
SVM,0.854749,0.877193,0.724638,0.793651
Decision Tree,0.782123,0.714286,0.724638,0.719424
Random Forest,0.826816,0.787879,0.753623,0.770370


### Inference

The comparison of machine learning models for Titanic survival prediction highlights some key insights. 

1. **Accuracy**: SVM achieves the highest accuracy at 85.47%, closely followed by Random Forest at 81.01%. This indicates that SVM has the best overall correct prediction rate, suggesting its effectiveness in classifying passengers as survivors or non-survivors.

2. **Precision**: SVM also outperforms the other models in precision, with a score of 87.72%. This indicates that SVM is highly accurate when it predicts survival; it minimizes false positives, which is crucial in this context as it means fewer non-survivors being misclassified as survivors.

3. **Recall**: SVM again demonstrates strength in recall, indicating its ability to identify a significant portion of actual survivors among all survivors in the dataset. It achieves a recall of 72.46%, which means it captures a substantial portion of survivors.

4. **F1 Score**: SVM maintains its superiority in the F1 score, reflecting a balanced trade-off between precision and recall. It achieves an F1 score of 79.37%, which is indicative of its robust performance in predicting survival.

In summary, SVM emerges as the top-performing model in this context, excelling in accuracy, precision, recall, and F1 score. This suggests that SVM is a strong candidate for predicting Titanic survival based on the provided dataset, offering valuable insights into the factors influencing passenger survival rates.